In [1]:
from config import config

In [2]:
%load_ext autoreload
%autoreload 2
from model_utils.thswad import THWADModel, load_model_from_transe
from model_utils.trainer import THSWADTrainer
from utils.ds import *

In [3]:
import pickle
with open(config['wikidata']['transe'], "rb") as f:
    transe = pickle.load(f)


In [4]:
import json
with open('dataset/el_single_results_test.json', 'r') as f:
    el_single_results = json.load(f)

In [5]:
ds_ = DialogDataset(el_single_results['entity_ids'], 'pad', 5)
items2id = {}
for i, (prev, next_id) in enumerate(ds_):
    for item in prev:
        if item not in items2id and item != 'pad':
            items2id[item] = len(items2id)
    if next_id not in items2id and next_id != 'pad':
        items2id[next_id] = len(items2id)
id2items = list(items2id.keys())
ds = DialogDataset(el_single_results['entity_ids'], len(items2id), 5, items2id)
ds[0]

/Users/panesher/code/dream-entity-recommendation/utils/ds.py:64: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all(np.array(prev) == pad_item):


(array([574, 574, 574, 574,   0]), 1)

In [6]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=5, shuffle=False)
for x in dl:
    break
x[0][0]

tensor([574, 574, 574, 574,   0])

In [7]:
wds = load_wiki_dataset(config['wikidata']['dataset']['test'], transe.graph)
len(wds)

144 triplets were removed from dataset/wikidata5m_transductive_test.txt


4989

In [8]:
wdl = DataLoader(wds, batch_size=5, shuffle=False)

In [9]:
model = load_model_from_transe(transe, id2items, prev_items_total=5)

In [12]:
loader = TSHWADLoader(wdl, dl)

In [13]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(opt, 1, gamma=0.9)

trainer = THSWADTrainer(
    margin=1.,
    norm_lambda=1.,
    clipping_max_value=1.,
    model_target=1.,
    kg_lambda=1.,
    model=model,
    optimizer=opt,
    scheduler=scheduler,
    train_data_loader=loader,
    val_data_loader=loader,
    test_data_loader=loader,
    verbose=False,
)

In [22]:
trainer.train(2)

Training: 100%|█████████▉| 1986/1991 [00:13<00:00, 146.28it/s]


In [23]:
trainer.test(loader)

Validating: 100%|█████████▉| 1986/1991 [00:10<00:00, 185.91it/s]


{'accuracy': tensor(0.0040)}